<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_forecasting/blob/main/LSTM_AUSTRALIA_10_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import csv



Mounted at /content/drive


#Parameters to change

In [ ]:
#number of houses
n_house = 50
#number of days
n_days = 10
#minumum delta for early stopping 
early_stop_delta = 0.0001

# SUBSET: draw random houses

In [ ]:
path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Australia_numpy_removed_night_hours.npy'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance_AUS(2012-2013).csv'

In [ ]:
import random

random.seed(42)

subset_col=[]
while True:
  r=random.randint(0,298)
  if r not in subset_col:
    subset_col.append(r)
  if len(subset_col) ==n_house:
    break

In [ ]:
array = np.loadtxt(path1)
print("Shape before reshape",np.shape(array))

array = array[:,subset_col]
print(array.shape)

Shape before reshape (6570, 299)
(6570, 50)


In [ ]:
AUS_data = array.reshape(array.shape[0]*array.shape[1], order="F")
print("Shape after reshape",np.shape(AUS_data))

Shape after reshape (328500,)


In [ ]:
df2=read_csv(path2, header =0, index_col=0) 
df2.shape
df2=df2.drop("time", axis=1)
array2 = df2.values
weath_data = array2.reshape(array2.shape[0]*array2.shape[1])
print("Shape of Aus data = ", AUS_data.shape, "\n","Shape of weather data = ", weath_data.shape )


Shape of Aus data =  (328500,) 
 Shape of weather data =  (6570,)


In [ ]:
#stacking irradiance as many times as there are houses in the dataset to match with each house in the datset

weath_data_stacked = np.tile(weath_data,n_house)
weath_data_stacked.shape
data= np.column_stack((AUS_data,weath_data_stacked))
data.shape
#data = np.nan_to_num(data)


(328500, 2)

In [ ]:
from numpy import split
from numpy import array
from keras.callbacks import ModelCheckpoint


# split a multivariate dataset into train/test sets
def split_dataset(data):
  # split 
  train, test = data[:-18], data[-18:]
  return train, test

def restructure_windows(train,test): 
  # restructure into windows
  train = array(split(train, len(train)/18))
  test = array(split(test, len(test)/18))
  return train, test

def scaling(train,test):
  #scaling values
  scalers={}

  scaled_train = []
  for i in range(train.shape[1]):
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[:,i].reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ str(i)] = scaler
    scaled_train.append(s_s)
  
  scaled_test = []
  for i in range(train.shape[1]):
    scaler = scalers['scaler_'+str(i)]
    s_s = scaler.transform(test[:,i].reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+str(i)] = scaler
    scaled_test.append(s_s)
  
  scaled_train = array(scaled_train)
  scaled_train=scaled_train.transpose()

  scaled_test = array(scaled_test)
  scaled_test=scaled_test.transpose()
  return scaled_train,scaled_test,scalers

# evaluate hours forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scores = list()
  # calculate an RMSE score for each day
  for i in range(actual.shape[1]):
    # calculate mse
    mse = mean_squared_error(actual[:, i], predicted[:, i])
    # calculate rmse
    rmse = sqrt(mse)
    # store
    scores.append(rmse)
  # calculate overall RMSE
  s = 0
  for row in range(actual.shape[0]):
    for col in range(actual.shape[1]):
      s += (actual[row, col] - predicted[row, col])**2
  score = sqrt(s / (actual.shape[0] * actual.shape[1]))
  return score, scores

# summarize scores
def summarize_scores(name, score, scores):
  s_scores = ', '.join(['%.1f' % s for s in scores])
  print('%s: [%.3f] %s' % (name, score, s_scores))
  
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=18):
  # flatten data
  data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
  X, y = list(), list()
  in_start = 0
  # step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    # Here we are dividing the data differently because we are working with a multivariate dataset (It contains more variables) 
    if out_end < len(data):
      X.append(data[in_start:in_end, :])
      y.append(data[in_end:out_end, 0])
    # move along one time step
    in_start += 1
  return array(X), array(y)

# train the model
def build_model(train, n_input):
  # prepare data
  train_x, train_y = to_supervised(train, n_input)
  # define parameters
  verbose, epochs, batch_size = 1, 20, 54
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # # reshape into subsequences [samples, timesteps, rows, cols, channels]
  # train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
  # # reshape output into [samples, timesteps, features]
  # train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  # stop when loss remains the same for 2 epochs --> passed to the model.fit
  early_stopping = EarlyStopping(monitor='loss', patience=2,min_delta=early_stop_delta,restore_best_weights=True)

  # define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', input_shape=(n_timesteps, n_features)))
  model.add(RepeatVector(n_outputs))
  model.add(LSTM(100, activation='relu', return_sequences=True))
  #model.add(LSTM(20, activation='relu', return_sequences=True))
  model.add(TimeDistributed(Dense(100, activation='relu')))
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss='mse', optimizer='adam')

  #saves the model on drive. Because of the loop (CV), one more parameter was passed (nth_house)
  path = '/content/drive/MyDrive/Master_thesis/Models/LSTM_house_'+str(n_days)+'_days_'+str(n_house)+'_houses_ALL_test.h5'
  model_checkpoint = ModelCheckpoint(path, monitor='loss', mode='min', verbose=1, save_best_only=True)
  # fit network
  model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,model_checkpoint], verbose=verbose, shuffle=False)
  return model


# make a forecast
def forecast(model, history,  n_input):
  # flatten data
  data = array(history)
  data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
  # retrieve last observations for input data
  input_x = data[-n_input:, :]
  # reshape into [1, n_input, n]
  input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
  # forecast the next 
  yhat = model.predict(input_x, verbose=0)
  # we only want the vector forecast
  yhat = yhat[0]
  return yhat

# evaluate a single model
def evaluate_model(train_scaled, test, n_input,test_scaled,scaler_fitted): 
  # fit model
  model = build_model(train_scaled,  n_input)
  history = [x for x in train_scaled]
  # walk-forward validation over each 5 days
  predictions = list()
  for i in range(len(test_scaled)):
    # predict the next day
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next 
    history.append(test_scaled[i, :])
  # evaluate predictions 
  predictions = array(predictions) 
  #print("shape predictions not inverse scaled", predictions.shape)
  #print("predictions",predictions)
  #inverse predictions 
  predictions =  predictions[:,:,0]
  predictions_inverse_scaled = scaler_fitted.inverse_transform(predictions)
  # sometimes there are very small negative numbers. These will be treated as zero
  predictions_inverse_scaled = np.abs(predictions_inverse_scaled)
  #evaluate with test data and inverse_scaled predicitions
  score, scores = evaluate_forecasts(test[:, :, 0], predictions_inverse_scaled)
  #print("prediction shape =",predictions_inverse_scaled.shape)
  #print("predictions_inverse_scaled", predictions_inverse_scaled)
  #print("Shape of test[:,:,0]", test[:, :, 0].shape)
  return score, scores,predictions_inverse_scaled

def random_cv(array1,nth_house,length):
  house = array1[(nth_house-1)*length: nth_house*length]
  before_house = array1[:(nth_house-1)*length]
  after_house = array1[nth_house*length:]
  return np.concatenate((before_house,after_house,house))
  


In [ ]:
#for i in random_houses:
for i in range(3,n_house+1):
  data = random_cv(data,i,6570)
  #data = pd.DataFrame(data, columns = ["solar_output","irradiance"])
  # split into train and test
  train, test = split_dataset(data)
  #scaling train,test
  train_scaled,test_scaled, scalers = scaling(train,test)
  #restructure test into windows without scaling. Non-scaled test is needed to evaluate forecast (passing it to "evaluate_model" function)
  _,unscaled_test = restructure_windows(train,test)
  #restructure into windows
  train_structured,test_structured = restructure_windows(train_scaled,test_scaled)
  #define the number of subsequences and the length of subsequences
  #n_days, n_length = 5, 18
  n_length = 18
  # define the total days to use as input
  n_input = n_length * n_days
  scaler = scalers['scaler_0']
  score, scores,predictions = evaluate_model(train_structured, unscaled_test, n_input,test_structured,scaler)
  with open('/content/drive/MyDrive/Master_thesis/Results/LSTM_Overall_RMSEs_AUS_'+str(n_house)+'_houses_'+str(n_days)+'_days_ALL_test.csv','a') as fd:
    write = csv.writer(fd)
    write.writerow([score])
  with open('/content/drive/MyDrive/Master_thesis/Results/LSTM_RMSEs_per_hour_AUS_'+str(n_house)+'_houses_'+str(n_days)+'_days_ALL_test.csv','a') as nd:
    write_n = csv.writer(nd)
    write_n.writerow(scores)
  # summarize scores
  summarize_scores('LSTM', score, scores)


Epoch 1/10
6080/6080 [==============================] - ETA: 0s - loss: 0.0031
Epoch 00001: loss improved from inf to 0.00312, saving model to /content/drive/MyDrive/Master_thesis/Models/ConvLSTM_house_10_days_50_houses_3.h5
6080/6080 [==============================] - 775s 127ms/step - loss: 0.0031
Epoch 2/10
6080/6080 [==============================] - ETA: 0s - loss: 0.0027
Epoch 00002: loss improved from 0.00312 to 0.00272, saving model to /content/drive/MyDrive/Master_thesis/Models/ConvLSTM_house_10_days_50_houses_3.h5
6080/6080 [==============================] - 779s 128ms/step - loss: 0.0027
Epoch 3/10
6080/6080 [==============================] - ETA: 0s - loss: 0.0026
Epoch 00003: loss improved from 0.00272 to 0.00264, saving model to /content/drive/MyDrive/Master_thesis/Models/ConvLSTM_house_10_days_50_houses_3.h5
6080/6080 [==============================] - 782s 129ms/step - loss: 0.0026
prediction shape = (1, 18)
predictions_inverse_scaled [[0.06386777 0.08751021 0.04949504 

In [ ]:
path = '/content/drive/MyDrive/Master_thesis/Results/Overall_RMSEs_AUS_50_houses_10_days.csv'
rmse40=read_csv(path, header =None, index_col=False)
rmse40.mean()